In [5]:
!pip install contractions

  Using cached https://files.pythonhosted.org/packages/93/f4/0ec4a458e4368cc3be2c799411ecf0bc961930e566dadb9624563821b3a6/contractions-0.0.52-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d3/fe/021d7d76961b5ceb9f8d022c4138461d83beff36c3938dc424586085e559/textsearch-0.0.21-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6d/7b/19437c9a5bd16e1bb3a5bf43f7655e341882befceae0122e43c8e2c21e1e/anyascii-0.3.0-py3-none-any.whl


The system cannot find the path specified.


In [1]:
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('words')

In [6]:
import re
import pickle
import numpy as np
import pandas as pd
import contractions
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer

Using TensorFlow backend.


In [7]:
df_1 = pd.read_csv("../Dataset/csv/train.csv", header=0, index_col=0)
df_1 = df_1.drop(['title','author'], axis = 1)
df_1.dropna(inplace = True)
#print(df_1.isnull().sum(axis = 0))

#Expanding Contraction, Lower Case and Word Splitting
df_1['text'] = df_1['text'].apply(lambda x: [contractions.fix(word, slang=False).lower() for word in x.split()])

#Removing Punctuations
df_1['text'] = df_1['text'].apply(lambda x: [re.sub(r'[^\w\s]','', word)  for word in x])

#Removing Stop Words
stop_words = set(stopwords.words('english'))
df_1['text'] = df_1['text'].apply(lambda x: [word for word in x if word not in stop_words])

#Removing Special Charecter and Numbers
df_1['text'] = df_1['text'].apply(lambda x: [word for word in x if re.search("[@_!#$%^&*()<>?/|}{~:0-9]", word) == None])

#Removing Non-English Words
english_words = set(nltk.corpus.words.words())
df_1['text'] = df_1['text'].apply(lambda x: [word for word in x if word in english_words])

#Concating Words back to Sentence
df_1['text'] = df_1['text'].apply(lambda x: ' '.join(x))

#print(df_1.head(3))

In [8]:
df_2 = pd.read_csv("../Dataset/csv/test.csv", header=0, index_col=0)
df_2 = df_2.drop(['title','author'], axis = 1)
df_2.dropna(inplace = True)
#print(df_2.isnull().sum(axis = 0))

#Expanding Contraction, Lower Case and Word Splitting
df_2['text'] = df_2['text'].apply(lambda x: [contractions.fix(word, slang=False).lower() for word in x.split()])

#Removing Punctuations
df_2['text'] = df_2['text'].apply(lambda x: [re.sub(r'[^\w\s]','', word)  for word in x])

#Removing Stop Words
stop_words = set(stopwords.words('english'))
df_2['text'] = df_2['text'].apply(lambda x: [word for word in x if word not in stop_words])

#Removing Special Charecter and Numbers
df_2['text'] = df_2['text'].apply(lambda x: [word for word in x if re.search("[@_!#$%^&*()<>?/|}{~:0-9]", word) == None])

#Removing Non-English Words
english_words = set(nltk.corpus.words.words())
df_2['text'] = df_2['text'].apply(lambda x: [word for word in x if word in english_words])

#Concating Words back to Sentence
df_2['text'] = df_2['text'].apply(lambda x: ' '.join(x))

#Transforming Labels
df_2['label'] = df_2['label'].apply(lambda x: int(x))

#print(df_2.head(3))

In [9]:
tfidfVectorizer = TfidfVectorizer(use_idf=True,stop_words='english')
tfidfVectorizer.fit(df_1['text'])  
tfidfVectorizer.fit(df_2['text']) 
#print(tfidfVectorizer.get_feature_names())

trainTfidfVector = tfidfVectorizer.transform(df_1['text']) 
testTfidfVector = tfidfVectorizer.transform(df_2['text']) 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_1['text'])
tokenizer.fit_on_texts(df_2['text'])

trainSequence = np.array([np.array(seq) for seq in pad_sequences(tokenizer.texts_to_sequences(df_1['text']))])
testSequence = np.array([np.array(seq) for seq in pad_sequences(tokenizer.texts_to_sequences(df_2['text']))])

pickle.dump(tfidfVectorizer, open("../Dataset/tfidf_vectorizer-1.pickle", "wb"))
pickle.dump(tokenizer, open("../Dataset/tokenizer-1.pickle", "wb"))

pickle.dump(trainTfidfVector, open("../Dataset/tfidf_train-1.pickle", "wb"))
pickle.dump(trainSequence, open("../Dataset/sequence_train-1.pickle", "wb"))
pickle.dump(df_1['label'], open("../Dataset/label_train-1.pickle", "wb"))

pickle.dump(testTfidfVector, open("../Dataset/tfidf_test-1.pickle", "wb"))
pickle.dump(testSequence, open("../Dataset/sequence_test-1.pickle", "wb"))
pickle.dump(df_2['label'], open("../Dataset/label_test-1.pickle", "wb"))


In [10]:
tfidfVectorizer = pickle.load(open("../Dataset/tfidf_vectorizer-1.pickle", "rb"))
#print(tfidfVectorizer.get_feature_names())

tokenizer = pickle.load(open("../Dataset/tokenizer-1.pickle", "rb"))
#print(tfidfVectorizer.get_feature_names())

trainTfidfVector = pickle.load(open("../Dataset/tfidf_train-1.pickle", "rb"))
df_train = pd.DataFrame(data = trainTfidfVector.toarray(),columns = tfidfVectorizer.get_feature_names())
#print(df_train)

testTfidfVector = pickle.load(open("../Dataset/tfidf_test-1.pickle", "rb"))
df_test = pd.DataFrame(data = testTfidfVector.toarray(),columns = tfidfVectorizer.get_feature_names())
#print(df_test)

trainSequence = pickle.load(open("../Dataset/sequence_train-1.pickle", "rb"))
#print(trainSequence)

testSequence = pickle.load(open("../Dataset/sequence_test-1.pickle", "rb"))
#print(testSequence)

trainLabels = pickle.load(open("../Dataset/label_train-1.pickle", "rb"))
#print(trainLabels)

testLabels = pickle.load(open("../Dataset/label_test-1.pickle", "rb"))
#print(testLabels)